# 판례 유사도 비교
- 사용자의 입력과 판례의 유사도를 비교
- 가장 유사도가 높은 순서로 보여줌

## 1. 패키지 로드

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle as pk

from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## 2. Pre-Trained Ko-BERT 모델 로드

In [2]:
with tf.device("/device:GPU:0"):
    model = SentenceTransformer('jhgan/ko-sbert-sts')

## 3. 판례 데이터 로드

### 1) 판례 Origin Data 로드

In [3]:
prec_df = pd.read_csv("./data/prec_data_refine.csv", encoding='utf-8-sig')
prec_df.head()

C:\Users\Park Ji Seong\anaconda3\envs\nlp-gpu\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,2022-04-15,선고,수원고법,400202,형사,400102,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,없음,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,2022-04-11,자,서울고등법원,400202,가사,400103,결정,미기재,미기재,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",없음,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,2022-03-23,선고,대구고법,400202,민사,400101,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",없음,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,2022-03-22,자,대법원,400201,가사,400103,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),미기재,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",없음,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,2022-03-17,선고,대법원,400201,민사,400101,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n","대법원 2018. 11. 29. 선고 2016다266606, 266613 판결(공2...","【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


### 2) 판례 임베딩 Pickle Data 로드

In [5]:
with open("./data/prec_data_refine_embeddings.pickle", "rb") as fr:
    prec_embeddings = pk.load(fr)

## 4. 사용자의 판례 유사도 비교

### 1) 사용자의 입력

In [7]:
print("""<< 판례 유사도 비교 >>""")
user_input = input("사건 내용 입력 : ")

<< 판례 유사도 비교 >>
사건 내용 입력 : 사건 내용은 1년넘게 사귄 남자친구가 있었습니다 그 남자친구가 바람피고 저한테 걸려서 헤어지는과정에 말다툼을했었는데, 저한테 마주치면 코뼈부쉰다,팬다 어쩐다 협박을 했었어요. 몇일뒤 지나가는길에 그 남자친구를 마주쳤습니다 남자친구와 그의 친구들 총 5명이 있었는데 저한테 얘기를 하자고하길래 친구들있으니 저쪽가서 이야기하자 라고 하고 자리를 옮겼어요. (옮겼지만 그 친구들은 다시 남자친구와 제가 있는쪽으로 와서 구경했고요) 자리 옮기고 나서 얘기를 하려고 하니 얘기는커녕 그때 무슨생각으로 자기한테 욕을했냐 라면서 따지기 시작했고 일방적으로 욕을 먹기 시작했습니다. 얘기중에 손으로 제 팔 내려치길래 저는 그애가 분노조절도 못하고 충동적인 행동을 하는것을 잘 알기에 진정해라,이래서 너한테 좋을거없다 라고 수차례 얘기했음에도 자기 화를 이기지못해 저를 폭행했습니다. 머리를 손으로 내려치고 발로 팔을 차서 핸드폰 날라가서 망가지고, 제가 걔를 쳐다보니 뭘보냐면서 안면상그대로 손으로 맞았습니다 그러다 같이있던 제 친구가 112에 신고를하고 경찰분들 오기를 기다렸어요 그러다 사건접수하고 전 119타고 대학병원응급실가서 검사와 약처방받고 집으로 갔습니다. 경찰서까지 넘어가서 조사받고 단순 폭행으로 검찰 넘어가는중이에요


### 2) 사용자의 입력 임베딩

In [8]:
user_input_embeddings = model.encode(user_input)

### 3) 판례와 유사도 비교

In [12]:
prec_similarity = list(enumerate(cosine_similarity([user_input_embeddings], prec_embeddings)[0])) # 인덱스와 그 값으로 분리

In [14]:
# 점수가 높은 순으로 정렬
prec_similarity = sorted(prec_similarity, key = lambda x : x[1], reverse = True)

### 4) 사용자의 확인할 판례 수 입력

In [17]:
view_num = int(input(f"확인할 판례의 수를 입력하세요(최대 : {len(prec_similarity)}) : "))

확인할 판례의 수를 입력하세요(최대 : 83318) : 5


### 5) 유사도 상위 5개의 판례 보여주기

In [21]:
for i in range(view_num):
    prec_idx = prec_similarity[i][0]
    prec_sim = round(prec_similarity[i][1] * 100, 2)
    
    print(f"<< {i + 1} 순위 판례(유사도 : {prec_sim}%) >>")
    print(f"[사건 종류] : {prec_df['사건종류명'][prec_idx]}")
    print(f"[판례 내용] : \n{prec_df['판례내용'][prec_idx]}")

<< 1 순위 판례(유사도 : 56.55%) >>
[사건 종류] : 형사
[판례 내용] : 
【피 고 인】 【주    문】  피고인은 무죄【이    유】  1. 이 사건 공소사실은, 피고인은 농업에 종사하는 자로서, 피해자인 처가 요즘들어 갑자기 부부관계를 요구하면 거절할 뿐 아니라 노골적으로 싫은 소리를 하므로 이는 피해자가 약 5년전에 불륜의 관계를 맺은 적이 있는 같은 마을에 사는 청년 
        공소외 1과 지금도 그 관계를 유지하고 있기 때문이라고 생각하고 
        공소외 1과의 관계를 추궁하여 오던중, 1986.5.10. 09:00경부터 경남 의창군 내서면 삼계부락 속칭 소구리골내 피고인 소유의 배나무밭에서 2홉들이 소주 1병을 다 마시고 구덩이 파는 작업을 하던중 피해자와 
        공소외 1과의 관계를 생각하니 분통이 터져 그 관계를 추궁할 생각으로 작업을 멈추고 집으로 내려가 수도가에서 같은날 오후에 마산에 내다 팔 열무단을 다듬고 있던 피해자에게 배나무밭 일을 도와 달라고 이야기하여 먼저 보낸 뒤 사실대로 이야기하지 아니하면 죽여버리겠다고 다짐하면서 청산가리를 준비하여 같은날 11:40경 위 배나무밭에서 또 2홉들이 소주 1병을 마시고 나서 피해자에게 
        공소외 1과의 관계를 사실대로 이야기하라고 추궁하였으나 동인이 뉘우치기는커녕 "시동생 
        공소외 2도 내 가슴에 손을 대드라, 다른 동네에는 별 희한한 일이 있어도 그냥 살더라"는등 오히려 시동생 흉을 보고 대들므로 격분하여 살해할 것을 결의하고 피해자를 밀어 넘어뜨린 뒤 왼손으로 피해자의 입을 벌려 청산가리를 먹일려고 하였으나 동인이 손을 물어 먹이지 못하게 되자 다시 살려달라고 애원하는 피해자의 목을 왼손으로 힘껏 눌러 동인이 설골탈골골절로 가사상태에 있는 틈을 이용하여 소지하고 있던 청산가리를 먹여 청산염 중독으로 사망에 이르게 하여 살해한 것이라고 함에 있는 바, 피고인은 경찰이래 이 법정에 이르기까지 이 사건 범행을 극력 부인하면서 사건당일